# ------------------------------------------------------------
# Projekat: Klasifikacija naslova proizvoda
# Autorka: Vesna Sibinović
# Opis: U ovoj svesci analiziraću skup podataka o proizvodima
#       i treniraću modele za klasifikaciju naslova.
# ------------------------------------------------------------

In [ ]:
# Uvozim osnovne biblioteke koje ću koristiti kroz ceo projekat

import os
import pandas as pd        # za rad sa tabelarnim podacima
import numpy as np         # za numeričke operacije
import matplotlib.pyplot as plt  # za crtanje grafika
import seaborn as sns      # za estetski lepše vizualizacije
# Uvozim neophodne biblioteke za obradu teksta
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Učitavanje podataka:

In [ ]:
# Učitavam podatke o proizvodima iz CSV fajla
# Fajl se nalazi u folderu "data"

# Ovde upisujem apsolutnu putanju do svog products.csv fajla
putanja_fajla = r"D:\It-akademija\Introduction to Machine Learning using Python\Tasks\Vesna_Sibinovic_Task3\data\products.csv"

# Provera da li fajl postoji
if os.path.exists(putanja_fajla):
    data = pd.read_csv(putanja_fajla)
    print("Fajl je uspešno učitan!")
    display(data.head())  # prikaz prvih 5 redova
    print("Kolone u dataset-u:", data.columns)
else:
    print(f"Fajl nije pronađen! Proveri putanju: {putanja_fajla}")

# Osnovna provera

In [ ]:
# Proveravam osnovne informacije o podacima
data.info()

# Proveravam da li postoje nedostajuće vrednosti
data.isnull().sum()

# Plan rada

1. Učitati i pregledati podatke  
2. Očistiti i pripremiti skup za modelovanje  
3. Izvršiti osnovnu analizu karakteristika naslova  
4. Primeniti različite modele klasifikacije (npr. Logistic Regression, Naive Bayes, SVM)  
5. Uporediti rezultate i izabrati najbolji model  
6. Zaključak i preporuke

### 1. PRIPREMA PODATAKA ZA MODELOVANJE

In [ ]:
# ------------------------------------------------------------
# 1. PRIPREMA PODATAKA ZA MODELOVANJE
# ------------------------------------------------------------
# U ovom koraku ću očistiti tekstove i podeliti podatke na 
# trening i test skup. Cilj je da model dobije čiste, 
# standardizovane naslove proizvoda za učenje.
# ------------------------------------------------------------

# Ovde upisujem naziv kolone za Title
naziv_kolone = 'Product Title'

# Čistim tekst - uklanjam brojeve, znakove interpunkcije i prebacujem u mala slova
def ocisti_tekst(tekst):
    tekst = re.sub(r'[^a-zA-Z\s]', '', str(tekst)) # zadržavam samo slova i razmake
    tekst = tekst.lower()                          # prebacujem sve u mala slova
    tekst = re.sub(r'\s+', ' ', tekst).strip()     # uklanjam višestruke razmake
    return tekst

# Primena funkcije na kolonu sa naslovima
data['clean_title'] = data[naziv_kolone].apply(ocisti_tekst)
display(data[['clean_title']].head())

# Primena funkcije na kolonu sa naslovima
data['clean_title'] = data[naziv_kolone].apply(ocisti_tekst)

# Uklanjam vodeće i završne razmake u imenima kolona
data.columns = data.columns.str.strip()

# Proverim da li kolone postoje
print("Kolone u dataset-u:", data.columns)

# Ovde upisujem naziv kolone sa kategorijama
naziv_kolone_kategorija = 'Category Label'

# Čistim X i y od praznih vrednosti
data = data.dropna(subset=[naziv_kolone, naziv_kolone_kategorija])

X = data['clean_title']
y = data[naziv_kolone_kategorija]

# Delim podatke na trening i test skup (80% : 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # čuvam raspodelu klasa u oba skupa
)

print(f"Broj primera u trening skupu: {len(X_train)}")
print(f"Broj primera u test skupu: {len(X_test)}")

### 2. Vektorizacija teksta (pretvaranje u numeričke vrednosti)

In [ ]:
# ------------------------------------------------------------
# 2. VEKTORIZACIJA TEKSTA
# ------------------------------------------------------------
# Modelima mašinskog učenja su potrebni brojevi, ne reči.
# Zato koristim TF-IDF (Term Frequency – Inverse Document Frequency)
# koji meri koliko je neka reč važna u odnosu na sve dokumente.
# ------------------------------------------------------------

# Inicijalizujem TF-IDF vektorizator
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

# Učim vektorizator na trening skupu i transformišem tekstove u brojeve
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("Oblik matrice trening skupa:", X_train_tfidf.shape)
print("Oblik matrice test skupa:", X_test_tfidf.shape)

### 3. Treniranje i evaluacija modela

In [ ]:
# ------------------------------------------------------------
# 3. TRENING I EVALUACIJA MODELA
# ------------------------------------------------------------
# Treniramo tri klasifikatora i upoređujemo rezultate.
# ------------------------------------------------------------

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

# ------------------------------------------------------------
# Definicija modela sa kratkim i doslednim nazivima
# ------------------------------------------------------------
modeli = {
    "LogReg": LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
    "NaiveBayes": MultinomialNB(),
    "SVM": LinearSVC(random_state=42, max_iter=5000)
}

# Skladištim rezultate u rečnik
rezultati = {}

# ------------------------------------------------------------
# Petlja za trening i evaluaciju modela
# ------------------------------------------------------------
for naziv, model in modeli.items():
    print(f"\nTreniram model: {naziv}")
    
    # Treniranje na TF-IDF trening skupu
    model.fit(X_train_tfidf, y_train)
    
    # Predikcija na test skupu
    y_pred = model.predict(X_test_tfidf)
    
    # Tačnost modela
    acc = accuracy_score(y_test, y_pred)
    print(f"Tačnost modela (accuracy): {acc:.4f}")
    
    # Detaljan izveštaj po klasama
    izvestaj = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    print(classification_report(y_test, y_pred, zero_division=0))
    
    # Čuvanje rezultata
    rezultati[naziv] = {
        "model": model,
        "accuracy": acc,
        "classification_report": izvestaj,
        "y_pred": y_pred
    }

print("\nTrening svih modela je završen.")

### 4. Uporedni prikaz rezultata

In [ ]:
# ------------------------------------------------------------
# 4. UPORЕDNА ANALIZA MODELA
# ------------------------------------------------------------
# U ovom koraku ću uporediti tačnost svih modela 
# i prikazati rezultate u obliku tabele.
# ------------------------------------------------------------

# Kreiram DataFrame iz rečnika rezultata
tabela_rezultata = pd.DataFrame({
    "Model": list(rezultati.keys()),
    "Tačnost": [rezultati[m]["accuracy"] for m in rezultati]
}).sort_values(by="Tačnost", ascending=False)

# Sortiram po tačnosti (najbolji model prvi)
tabela_rezultata = tabela_rezultata.sort_values(by="Tačnost", ascending=False)

# Prikazujem tabelu
print("Uporedna tačnost modela:")
display(tabela_rezultata)

### 5. Vizualizacija rezultata

In [ ]:
# ------------------------------------------------------------
# 5. VIZUALIZACIJA REZULTATA
# ------------------------------------------------------------
# Na osnovu prethodne tabele crtam stubičasti grafikon 
# koji prikazuje koliko je tačan svaki model.
# ------------------------------------------------------------

plt.figure(figsize=(8,5))

ax = sns.barplot(
    x="Model",
    y="Tačnost",
    data=tabela_rezultata,
    palette="pastel",
    ci=None,
    legend=False  # ovo uklanja warning
)
ax.set_title("Uporedna tačnost modela", fontsize=14)
ax.set_xlabel("Model")
ax.set_ylabel("Tačnost")
ax.set_ylim(0, 1)  # Skala od 0 do 1

# Dodavanje tačnog broja tačnosti iznad svake trake
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.4f}", 
                (p.get_x() + p.get_width() / 2., height),
                ha='center', va='bottom', fontsize=10)

plt.show()

### 6. Zaključak i izbor najboljeg modela

In [ ]:
# ------------------------------------------------------------
# 6. ZAKLJUČAK I IZBOR NAJBOLJEG MODELA
# ------------------------------------------------------------
# Na osnovu dobijenih tačnosti i F1 metrike odlučujem koji
# model daje najbolje rezultate za naš skup proizvoda.
# ------------------------------------------------------------

# Pronađem model sa najvećom tačnošću
najbolji_model_naziv = tabela_rezultata.iloc[0]["Model"]
najbolji_model = rezultati[najbolji_model_naziv]["model"]

print(f"Najbolji model je: {najbolji_model_naziv}")
print(f"Tačnost na test skupu: {rezultati[najbolji_model_naziv]['accuracy']:.4f}")

# ------------------------------------------------------------
# 6. HEATMAP F1 SCORE PO KLASAMA
# ------------------------------------------------------------
# Vizualizujem F1 score po klasama za svaki model
# ------------------------------------------------------------

# Kreiram DataFrame gde će redovi biti klase, a kolone modeli
f1_data = {}

for naziv_modela, rez in rezultati.items():
    # Uzimam f1-score po klasama iz classification_report
    cr = rez['classification_report']
    # Izostavljam 'accuracy', 'macro avg', 'weighted avg'
    f1_po_klasama = {k: v['f1-score'] for k, v in cr.items() if k not in ['accuracy', 'macro avg', 'weighted avg']}
    f1_data[naziv_modela] = f1_po_klasama

# Pretvaram u DataFrame i popunjavam NaN sa 0 (ako neka klasa nije predviđena)
df_f1 = pd.DataFrame(f1_data).fillna(0)

# Crtam heatmap
plt.figure(figsize=(12,8))
sns.set(font_scale=1)
ax = sns.heatmap(df_f1, annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={'label': 'F1 score'})
plt.title("F1 score po klasama i modelima", fontsize=16, weight='bold')
plt.ylabel("Klasa proizvoda", fontsize=12)
plt.xlabel("Model", fontsize=12)
plt.show()

### 7. Čuvanje modela za kasniju upotrebu

In [ ]:
# ------------------------------------------------------------
# 7. ČUVANJE NAJBOLJEG MODELA U .pkl FORMAT
# ------------------------------------------------------------
# Ovaj model kasnije mogu učitati u skriptu predict_category.py
# da predviđa kategorije novih proizvoda.
# ------------------------------------------------------------

import pickle

# Putanja do src foldera iz notebooks foldera
src_folder = "../src"

if not os.path.exists(src_folder):
    os.makedirs(src_folder)

# Odabir modela sa najvećom tačnošću
najbolji_model_naziv = max(rezultati, key=lambda m: rezultati[m]['accuracy'])
najbolji_model = rezultati[najbolji_model_naziv]['model']

print(f"Najbolji model je: {najbolji_model_naziv} sa tačnošću {rezultati[najbolji_model_naziv]['accuracy']:.4f}")

# Čuvanje modela
with open(os.path.join(src_folder, "final_model.pkl"), "wb") as f:
    pickle.dump(najbolji_model, f)

# Čuvanje TF-IDF vektorizatora
with open(os.path.join(src_folder, "vectorizer.pkl"), "wb") as f:
    pickle.dump(vectorizer, f)

print("Model i TF-IDF vektorizator su sačuvani u folderu 'src/' na root-u projekta")

# Zaključci

- Najbolji model za klasifikaciju naslova proizvoda je **SVM**.
- Postignuta tačnost na test skupu je 0.9575.
- Model je sačuvan u fajlu `final_model.pkl`.
- TF-IDF vektorizator je sačuvan u fajlu `vectorizer.pkl`.
- Ova dva fajla se kasnije koriste za interaktivno predviđanje kategorija novih proizvoda.
- Neke klase sa malim brojem primera (npr. `CPU`, `Mobile Phone`, `fridge`) nisu dobro predviđene, što pokazuje ograničenje modela.
- Preporučujem prikupljanje dodatnih podataka za retke klase i eventualno podešavanje hiperparametara ili korišćenje drugih tehnika vektorizacije (n-grami, embeddings) za poboljšanje performansi.
- Model je sada spreman za primenu u realnom okruženju za automatsko kategorizovanje novih proizvoda.